<a href="https://colab.research.google.com/github/ben900926/WatClaimCheck_implement/blob/main/watclaimcheck.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Original github 
[github](https://github.com/nxii/WatClaimCheck.git)

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Data collection

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data Cleaning

In [ ]:
!python drive/MyDrive/WatClaimCheck/dataset_generation/data_cleaning/generate_dataset.py

Dataset preprocess

# Data Preporcess

In [ ]:
#!/usr/bin/env python3

import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import json
import os
import sys
import subprocess
from configparser import ConfigParser
from urllib.parse import urlparse
from collections import Counter
from tqdm import tqdm
from nltk import sent_tokenize


def remove_irrelevant_tags(node):
    script_nodes = node.find_all('script')
    for script_node in script_nodes:
        script_node.decompose()
    
    style_nodes = node.find_all('style')
    for style_node in style_nodes:
        style_node.decompose()
        

def extract_altnews_review_article(review_html):
    
    soup = BeautifulSoup(review_html, 'html.parser')
    
    article_node = soup.select("#main > div.m-hentry-s.m-h-s.post > div > div")[0]
    
    assert article_node is not None, "Error! could not find the article text node!"
    
    t_node = article_node.find("div", id="enhancedtextwidget-12")
    if t_node is not None:
        t_node.decompose()
        
    t_node = article_node.find("div", id="jp-relatedposts")
    if t_node is not None:
        t_node.decompose()
    
    remove_irrelevant_tags(article_node)
    
    return article_node.get_text().strip()

def extract_factcheck_review_article(review_html):
    
    soup = BeautifulSoup(review_html, 'html.parser')
    
    article_node = soup.find('div', class_="entry-content")
    
    assert article_node is not None, "Error! could not find the article text node!"
    
    h5_sources = article_node.find("h5", string="Sources")
    if h5_sources is not None:
        prev_sibling_node = h5_sources.find_previous_sibling("p")
        if prev_sibling_node is not None:
            prev_sib_text = prev_sibling_node.get_text()
            if "Editor's note" in prev_sib_text or "This fact check is available at IFCN’s" in prev_sib_text:
                prev_sibling_node.decompose()
                
                prev_sibling_node = h5_sources.find_previous_sibling("p")
                if prev_sibling_node is not None:
                    prev_sib_text = prev_sibling_node.get_text()
                    if "Editor’s Note:" in prev_sib_text or "This fact check is available at IFCN’s" in prev_sib_text:
                        prev_sibling_node.decompose()
            
        source_p_nodes = h5_sources.find_next_siblings("p")
        for source_p_node in source_p_nodes:
            source_p_node.decompose()
    
        h5_sources.decompose()
    
    remove_irrelevant_tags(article_node)
    
    return article_node.get_text().strip()

def extract_politifact_review_article(review_html):
    
    soup = BeautifulSoup(review_html, 'html.parser')
    
    article_node = soup.find('article', class_="m-textblock")
    
    assert article_node is not None, "Error! could not find the article text node!"
    
    stf_node = article_node.find("div", class_="artembed")
    if stf_node is not None:
        stf_node.decompose()
    
    ffc_node = article_node.find("section", class_="o-pick")
    if ffc_node is not None:
        ffc_node.decompose()
        
    remove_irrelevant_tags(article_node)
    
    return article_node.get_text().strip()

def extract_snopes_review_article(review_html):
    
    soup = BeautifulSoup(review_html, 'html.parser')
    
    article_nodes = soup.select("div.single-body.card.card-body.rich-text")
    
    assert len(article_nodes) > 0, "Error! could not find the article text node!"
    
    article_node = article_nodes[0]
    
    remove_irrelevant_tags(article_node)
    
    return article_node.get_text().strip()

def extract_afp_review_article(review_html):
    
    soup = BeautifulSoup(review_html, 'html.parser')
    
    article_node = soup.find('div', class_="article-entry")
    
    assert article_node is not None, "Error! could not find the article text node!"
    
    remove_irrelevant_tags(article_node)
    
    return article_node.get_text().strip()

def extract_africacheck_review_article(review_html):
    
    soup = BeautifulSoup(review_html, 'html.parser')
    
    article_nodes = soup.select("#block-mainpagecontent > article > div > div.cell.medium-8.article--main")
    
    assert len(article_nodes) > 0, "Error! could not find the article text node!"
    
    article_node = article_nodes[0]
    
    remove_irrelevant_tags(article_node)
    
    return article_node.get_text().strip()

def extract_usatoday_review_article(review_html):
    
    soup = BeautifulSoup(review_html, 'html.parser')
    
    article_node = soup.find("article")
    
    assert article_node is not None, "Error! could not find the article text node!"
    
    remove_irrelevant_tags(article_node)
    
    return article_node.get_text().strip()
    
def extract_fullfact_review_article(review_html):
    
    soup = BeautifulSoup(review_html, 'html.parser')
    
    article_node = soup.find("article")
    
    assert article_node is not None, "Error! could not find the article text node!"
    
    remove_irrelevant_tags(article_node)
    
    return article_node.get_text().strip()

def extract_review_text(review_url, review_article_path):
    
    review_text_extractors = {
        "www.altnews.in": extract_altnews_review_article,
        "www.factcheck.org": extract_factcheck_review_article,
        "www.politifact.com": extract_politifact_review_article,
        "www.snopes.com": extract_snopes_review_article,
        "factcheck.afp.com": extract_afp_review_article,
        "africacheck.org": extract_africacheck_review_article,
        "www.usatoday.com": extract_usatoday_review_article,
        "fullfact.org": extract_fullfact_review_article
    }
    
    with open(review_article_path) as f_obj:
        article_html = f_obj.read()
    
    url_parts = urlparse(review_url)
    host = url_parts.netloc
    
    try:
        article_text = review_text_extractors[host](article_html)
    except Exception as e:
        print("Following exception occurred while extracting review article text!")
        print(e)
        print(f"Review article URL: {review_url}")
        print(article_html)
        sys.exit(1)
    
    article_sentences = sent_tokenize(article_text)
    article_sentences = [sent.strip() for sent in article_sentences if sent.strip() != '']
    
    return article_sentences
    
    
def load_articles(dataset_path, articles_path):
    
    with open(os.path.join(dataset_path, 'train.json')) as f_obj:
        train_records = json.load(f_obj)

    with open(os.path.join(dataset_path, 'valid.json')) as f_obj:
        valid_records = json.load(f_obj)
  
    with open(os.path.join(dataset_path, 'test.json')) as f_obj:  
        test_records = json.load(f_obj)
  
    all_records = train_records + valid_records + test_records

    review_articles = []
    for record in all_records:
      article_path = record["label"]["review_article_html_path"]
      article_path = article_path.split('\\')
      article_path = '/'.join(article_path)
      info = {
          "url": record["label"]["review_url"],
          "path": os.path.join('/content/drive/MyDrive/WatClaimCheck/paper_code', article_path)
      }
      p = info["path"]
      assert os.path.isfile(info["path"]) == True, f"Error! Review article json file does not exists: {p}"
      review_articles.append(info)
    
    rev_article_path_to_sents = {}
    all_rev_sentences = []
    hosts = []
    for rev_article in tqdm(review_articles):
        article_sentences = extract_review_text(rev_article["url"], rev_article["path"])
        file_path, file_name = os.path.split(rev_article["path"])
        file_pre, file_ext = os.path.splitext(file_name)
                
        sent_file_name = f"{file_pre}_sent.json"
        sent_file_path = os.path.join(file_path, sent_file_name)
        
        with open(sent_file_path, "w") as f_obj:
            json.dump(article_sentences, f_obj, indent=4)
            
        rev_article_path_to_sents[rev_article["path"]] = article_sentences
        
    return rev_article_path_to_sents


def load_data_w_review_sents(dataset_path, article_path_to_sents):
  with open(os.path.join(dataset_path, 'train.json')) as f_obj:
    train_records = json.load(f_obj)
  
  train = []
  for record in tqdm(train_records):
    train_item = {"id": record["metadata"]["id"]}
    
    if record['metadata']['claimant'] is not None:
      train_item['text'] = record['metadata']['claim'].strip() + ' ' + record['metadata']['claimant'].strip() 
      train_item['text'] = train_item['text'].strip()
    else:
      train_item['text'] = record['metadata']['claim'].strip()

    article_path = record["label"]["review_article_html_path"]
    p = article_path.split('\\')
    p = '/'.join(p)
    article_path = os.path.join('/content/drive/MyDrive/WatClaimCheck/paper_code', p)
    if article_path in article_path_to_sents:
        train_item["review_sentences"] = article_path_to_sents[article_path]
    else:
        continue
    
    train.append(train_item)
  
  train_df = pd.DataFrame.from_records(train)

  with open(os.path.join(dataset_path, 'valid.json')) as f_obj:
    valid_records = json.load(f_obj)

  valid = []
  for record in tqdm(valid_records):
    valid_item = {"id": record["metadata"]["id"]}

    if record['metadata']['claimant'] is not None:
      valid_item['text'] = record['metadata']['claim'].strip() + ' ' + record['metadata']['claimant'].strip() 
      valid_item['text'] = valid_item['text'].strip()
    else:
      valid_item['text'] = record['metadata']['claim'].strip()

    article_path = record["label"]["review_article_html_path"]
    p = article_path.split('\\')
    p = '/'.join(p)
    article_path = os.path.join('/content/drive/MyDrive/WatClaimCheck/paper_code', p)
    if article_path in article_path_to_sents:
        valid_item["review_sentences"] = article_path_to_sents[article_path]
    else:
        continue
    
    valid.append(valid_item)
  
  valid_df = pd.DataFrame.from_records(valid)


  with open(os.path.join(dataset_path, 'test.json')) as f_obj:
    test_records = json.load(f_obj)

  test = []
  for record in tqdm(test_records):
    test_item = {"id": record["metadata"]["id"]}

    if record['metadata']['claimant'] is not None:
      test_item['text'] = record['metadata']['claim'].strip() + ' ' + record['metadata']['claimant'].strip() 
      test_item['text'] = test_item['text'].strip()
    else:
      test_item['text'] = record['metadata']['claim'].strip()

    article_path = record["label"]["review_article_html_path"]
    p = article_path.split('\\')
    p = '/'.join(p)
    article_path = os.path.join('/content/drive/MyDrive/WatClaimCheck/paper_code', p)
    
    if article_path in article_path_to_sents:
        test_item["review_sentences"] = article_path_to_sents[article_path]
    else:
        continue
    
    test.append(test_item)
  test_df = pd.DataFrame.from_records(test)

  return train_df, valid_df, test_df


def main(config):
    
    dataset_path = config['default']['dataset_path']
    articles_path = os.path.join(dataset_path, config['default']['dataset_articles'])
    
    
    print('Loading articles...')
    # {'/content/drive/MyDrive/WatClaimCheck/paper_code/dataset/articles/1.html': ['Gov.', 'Glenn Youn
    rev_article_path_to_sents = load_articles(dataset_path, articles_path)
    # something's wrong here
    print('Generating dataframes with review article sentences...')
    train_w_rev, valid_w_rev, test_w_rev = load_data_w_review_sents(dataset_path, rev_article_path_to_sents)

    train_w_rev.to_pickle(os.path.join(dataset_path, 'train_w_review_sents.pkl'))
    valid_w_rev.to_pickle(os.path.join(dataset_path, 'valid_w_review_sents.pkl'))
    test_w_rev.to_pickle(os.path.join(dataset_path, 'test_w_review_sents.pkl'))
    
    
if __name__ == "__main__":
    config = dict()
    config['default'] = {
        'data_dir' : '/content/drive/MyDrive/WatClaimCheck/paper_code/data',
        'articles_dir' : 'articles',
        'data_sources' : 'Politifact',
        'metadata_filename' : 'claim_metadata_w_relevant.json',
        'min_articles_req' : 2,

        'dataset_path' : '/content/drive/MyDrive/WatClaimCheck/paper_code/dataset',
        'dataset_articles' : 'articles',

        'train_proportion' : 0.8,

        'train_file_w_review_sents' : 'train_w_review_sents.pkl'
    }
    main(config)


Loading articles...


100%|██████████| 7/7 [00:00<00:00,  7.84it/s]


Generating dataframes with review article sentences...


100%|██████████| 1/1 [00:00<00:00, 6472.69it/s]

[{'id': 7, 'text': 'Video shows Joe Biden has a body double. Viral image', 'review_sentences': ['A recent Facebook post showing a series of photos of President Joe Biden suggests that the images don’t show Biden, but rather a body double.', 'That’s wrong.', 'This is a longtime falsehood that we’ve looked at before, and it’s unfounded.', 'This post was flagged as part of Facebook’s efforts to combat false news and misinformation on its News Feed.', '(Read more about our partnership with Facebook.)', 'The post shows a series of side-by-side comparisons of two photos.', 'Both are Biden, and while he looks different in one from the next, it’s not because of a body-double scheme.', 'It’s because of aging.', 'In one such set of photos, Biden is pictured at a vice presidential debate in 2008 when he was in his mid-60s, and at a presidential debate in 2020, when he was in his late 70s.', '(Biden is now 79 years old.)', 'Another set of photos shows Biden in 2008 and in 2019.', 'Yet another set 

DPR

In [ ]:
!pip install transformers

In [ ]:
import pandas as pd
from transformers import RobertaTokenizer
from torch.utils.data import TensorDataset, DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import RobertaTokenizer, RobertaModel, AdamW
import numpy as np
import shutil
from tqdm import tqdm
from configparser import ConfigParser
import os
import json
import argparse
import shutil

def load_data(data_dir):

    df = {}
    df["train"] = pd.read_pickle(os.path.join(data_dir, "train_w_review_sents.pkl"))
    df["valid"] = pd.read_pickle(os.path.join(data_dir, "valid_w_review_sents.pkl"))
    df["test"] = pd.read_pickle(os.path.join(data_dir, "test_w_review_sents.pkl"))

    claim_ids = {}
    claim_txt = {}
    sentences = {}
    sent_ids = {}
    id2sents = {}
    id2claims = {}

    for partition in df.keys():
        claim_ids[partition] = []
        claim_txt[partition] = []
        sentences[partition] = []
        sent_ids[partition] = []
        id2sents[partition] = {}
        id2claims[partition] = {}

        for _, row in df[partition].iterrows():
            claim_ids[partition].append(row["id"])
            claim_txt[partition].append(row["text"])
            sentences[partition].extend(row["review_sentences"])
            sent_ids[partition].extend([row["id"]] * len(row["review_sentences"]))
            id2sents[partition][row["id"]] = row["review_sentences"]
            id2claims[partition][row["id"]] = row["text"]

    return claim_ids, claim_txt, sentences , sent_ids, id2claims, id2sents


def build_validation_data_loader(
    claim_ids, claim_txt, sentences , sent_ids, claim_msl, sent_msl, tokenizer
):

  partitions = ["valid", "test"]

  ct_input_ids = {}
  ct_dataset = {}
  ct_dataloader = {}
  for partition in partitions:
    ct_input_ids[partition] = tokenizer(claim_txt[partition], padding="max_length", truncation=True, max_length=claim_msl, return_tensors="pt").input_ids
    ct_dataset[partition] = TensorDataset(ct_input_ids[partition])
    ct_dataloader[partition] = DataLoader(ct_dataset[partition], shuffle=False, batch_size=64)

  sent_input_ids = {}
  sent_dataset = {}
  sent_dataloader = {}
  for partition in partitions:
    sent_input_ids[partition] = tokenizer(sentences[partition], padding="max_length", truncation=True, max_length=sent_msl, return_tensors="pt").input_ids
    sent_dataset[partition] = TensorDataset(sent_input_ids[partition])
    sent_dataloader[partition] = DataLoader(sent_dataset[partition], shuffle=False, batch_size=64)

  return ct_dataloader, sent_dataloader


def convert_to_tensors(
    train_id2claims,
    train_id2sents,
    tokenizer,
    claim_msl,
    sent_msl
):
  train_id2claims_tensor = {}
  train_id2sents_tensor = {}
  for id in train_id2claims.keys():
    if len(train_id2sents[id]) == 0:
      continue
    train_id2claims_tensor[id] = tokenizer(train_id2claims[id], padding="max_length", truncation=True, max_length=claim_msl, return_tensors="pt").input_ids
    train_id2sents_tensor[id] = tokenizer(train_id2sents[id], padding="max_length", truncation=True, max_length=sent_msl, return_tensors="pt").input_ids

  return train_id2claims_tensor, train_id2sents_tensor


def compute_recall(claim_vecs, sentence_vecs, claim_labels, sentence_labels, top_k_list):

    sentence_labels = np.array(sentence_labels)
    recalls = {top_k: list()  for top_k in top_k_list}
    for index in range(claim_vecs.shape[0]):
        claim_vec = claim_vecs[index]
        claim_label = claim_labels[index]

        sim_scores = sentence_vecs.dot(claim_vec)

        label_sent_indices = np.where(sentence_labels==claim_label)[0]

        sorted_indices = np.argsort(sim_scores)[::-1]

        total_relevant = len(label_sent_indices)
        for top_k in top_k_list:
            retrieved_relevant = len(np.intersect1d(sorted_indices[:top_k], label_sent_indices, assume_unique=True))
            recall = retrieved_relevant / total_relevant

            recalls[top_k].append(recall)

    return recalls

class SentenceSimilarityModel(nn.Module):

  def __init__(self):
    super().__init__()
    self.c_encoder = RobertaModel.from_pretrained('distilroberta-base')
    self.s_encoder = RobertaModel.from_pretrained('distilroberta-base')

  def embed_claims(self, claims):
    return self.c_encoder(claims).pooler_output

  def embed_sents(self, sents):
    return self.s_encoder(sents).pooler_output

  def forward(self, claims, sents):

    c_embeddings = self.c_encoder(claims).pooler_output
    s_embeddings = self.s_encoder(sents).pooler_output

    s_embeddings_t = s_embeddings.transpose(0, 1)

    sim_scores = torch.matmul(c_embeddings, s_embeddings_t)
    return sim_scores

def biencoder_nll_loss(sim_scores, labels):

  softmax_scores = F.log_softmax(sim_scores, dim=1)
  loss = F.nll_loss(softmax_scores, labels, reduction="none")

  max_score, max_idxs = torch.max(softmax_scores, 1)
  correct_predictions_count = (max_idxs == labels).sum()

  return loss, correct_predictions_count

def get_optimizer(
    model: nn.Module,
    learning_rate: float = 1e-5,
    adam_eps: float = 1e-8,
    weight_decay: float = 0.0,
) -> torch.optim.Optimizer:
    no_decay = ["bias", "LayerNorm.weight"]

    optimizer_grouped_parameters = [
        {
            "params": [
                p
                for n, p in model.named_parameters()
                if not any(nd in n for nd in no_decay)
            ],
            "weight_decay": weight_decay,
        },
        {
            "params": [
                p
                for n, p in model.named_parameters()
                if any(nd in n for nd in no_decay)
            ],
            "weight_decay": 0.0,
        },
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate, eps=adam_eps)
    return optimizer


def load_model_checkpoint(model_checkpoint_file):
    if not os.path.isfile(model_checkpoint_file):
        return None
    
    return torch.load(model_checkpoint_file)


def main(args):
    
    # Set manual seed
    torch.manual_seed(args.rand_seed)
    
    output_dir = os.path.join(args.output_dir, args.exp_name, f"Seed_{args.rand_seed}")
    if not os.path.isdir(output_dir):
        os.makedirs(output_dir)
    
    claim_ids, claim_txt, sentences , sent_ids, id2claims, id2sents = load_data(args.data_dir)

    tokenizer = RobertaTokenizer.from_pretrained('distilroberta-base')
    claim_msl = 120
    sent_msl = 320

    eval_ct_dataloader, eval_sent_dataloader = build_validation_data_loader(
        claim_ids, claim_txt, sentences , sent_ids, claim_msl, sent_msl, tokenizer
    )

    train_id2claims_tensor, train_id2sents_tensor = convert_to_tensors(
        id2claims["train"],
        id2sents["train"],
        tokenizer,
        claim_msl,
        sent_msl
    )
    train_dataset = TensorDataset(torch.tensor(list(train_id2claims_tensor.keys())).cuda())
    train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=args.batch_size)
    
    model = SentenceSimilarityModel().cuda()
    optimizer = get_optimizer(model)
    
    model_checkpoint_file = os.path.join(args.checkpoint_dir, "model.tar")
    temp_checkpoint_file = os.path.join(args.checkpoint_dir, "temp_model.tar")
    
    model_checkpoint = load_model_checkpoint(model_checkpoint_file)
    
    if model_checkpoint is not None:
        model.load_state_dict(model_checkpoint["model_state_dict"])
        optimizer.load_state_dict(model_checkpoint['optimizer_state_dict'])
        start_epoch = model_checkpoint["epoch"] + 1
        print(f"Resuming model training from epoch: {start_epoch+1}")
        
        eval_results_filename = f"eval_results_{epoch+1}.json"
        eval_results_file_path = os.path.join(output_dir, eval_results_filename)
        
        model_filename = f"model_{epoch+1}.pt"
        model_file_path = os.path.join(output_dir, model_filename)
        
        if ((epoch+1) % validate_every) == 0 and not (os.path.isfile(eval_results_file_path) and os.path.isfile(model_file_path)):
            print()
            print("Validating model.")
            model.eval()
            with torch.no_grad():
                ct_vecs = {}
                for partition in eval_ct_dataloader.keys():
                    ct_vecs[partition] = []
                    for batch in tqdm(eval_ct_dataloader[partition]):
                        ct_batch = batch[0].cuda()
                        embeddings = model.embed_claims(ct_batch)
                        ct_vecs[partition].append(embeddings.cpu())

                    ct_vecs[partition] = torch.cat(ct_vecs[partition]).numpy()

                sent_vecs = {}
                for partition in eval_sent_dataloader.keys():
                    sent_vecs[partition] = []
                    for batch in tqdm(eval_sent_dataloader[partition]):
                        sent_batch = batch[0].cuda()
                        embeddings = model.embed_sents(sent_batch)
                        sent_vecs[partition].append(embeddings.cpu())

                    sent_vecs[partition] = torch.cat(sent_vecs[partition]).numpy()

            top_k_list = [10, 25, 50, 100]
            eval_results = {}
            for partition in eval_ct_dataloader.keys():
                print(partition)
                recalls = compute_recall(ct_vecs[partition], sent_vecs[partition], claim_ids[partition], sent_ids[partition], top_k_list)
                eval_results[partition] = {}
                for top_k, recall in recalls.items():
                    eval_results[partition][top_k] = np.mean(recall)
                    print(f"Top-{top_k} Recall: {round(np.mean(recall),2)*100}")
                
            print()
            
            print(f"Saving eval results to {eval_results_file_path}")
            with open(eval_results_file_path, "w") as f_obj:
                json.dump(eval_results, f_obj, indent=4)
            
            print(f"Saving model to {model_file_path}")
            torch.save(model, model_file_path)
            print()
        
    else:
        start_epoch = 0
    
    log_every = 25
    validate_every = 10

    for epoch in range(start_epoch, 500):
        train_loss = []
        train_correct = 0
        train_total = 0
        model.train()
        with torch.set_grad_enabled(True):
            for index, batch in enumerate(train_dataloader):
                indices = batch[0]

                claims = []
                sents = []
                for id in indices:
                    id = id.item()
                    claims.append(train_id2claims_tensor[id].squeeze())
                    sents.append(train_id2sents_tensor[id][epoch%train_id2sents_tensor[id].size(0)])

                claims = torch.stack(claims).cuda()
                sents = torch.stack(sents).cuda()

                sim_scores = model(claims, sents)
                labels = torch.arange(len(indices)).cuda()

                loss, correct_predictions_count = biencoder_nll_loss(sim_scores, labels)

                train_correct += correct_predictions_count
                train_total += len(indices)

                train_loss.extend(loss.detach().cpu().tolist())

                batch_loss = torch.mean(loss)

                if ((index + 1) % log_every) == 0:
                    print(f"Epoch: {epoch+1} Batch# {index+1}, Batch loss: {batch_loss.item():.2f} Correct Predictions: {correct_predictions_count}")

                optimizer.zero_grad()
                batch_loss.backward()
                optimizer.step()

            print()
            print(f"Epoch # {epoch+1} Train Loss: {np.mean(train_loss).item():.2f} Correct Predictions: {(train_correct/train_total)*100:.2f}")
            print()
            
            
            try:
                torch.save(
                    {
                        'epoch': epoch,
                        'model_state_dict': model.state_dict(),
                        'optimizer_state_dict': optimizer.state_dict()
                    }, 
                    temp_checkpoint_file
                )
                save_successful = True
            except:
                print(f"Warning! Could not save model for epoch: {epoch+1}!")
                save_successful = False
            
            if save_successful:
                shutil.move(temp_checkpoint_file, model_checkpoint_file)
                

            if ((epoch+1) % validate_every) == 0:
                print()
                print("Validating model.")
                model.eval()
                with torch.no_grad():
                    ct_vecs = {}
                    for partition in eval_ct_dataloader.keys():
                        ct_vecs[partition] = []
                        for batch in tqdm(eval_ct_dataloader[partition]):
                            ct_batch = batch[0].cuda()
                            embeddings = model.embed_claims(ct_batch)
                            ct_vecs[partition].append(embeddings.cpu())

                        ct_vecs[partition] = torch.cat(ct_vecs[partition]).numpy()

                    sent_vecs = {}
                    for partition in eval_sent_dataloader.keys():
                        sent_vecs[partition] = []
                        for batch in tqdm(eval_sent_dataloader[partition]):
                            sent_batch = batch[0].cuda()
                            embeddings = model.embed_sents(sent_batch)
                            sent_vecs[partition].append(embeddings.cpu())

                        sent_vecs[partition] = torch.cat(sent_vecs[partition]).numpy()

                top_k_list = [10, 25, 50, 100]
                eval_results = {}
                for partition in eval_ct_dataloader.keys():
                    print(partition)
                    recalls = compute_recall(ct_vecs[partition], sent_vecs[partition], claim_ids[partition], sent_ids[partition], top_k_list)
                    eval_results[partition] = {}
                    for top_k, recall in recalls.items():
                        eval_results[partition][top_k] = np.mean(recall)
                        print(f"Top-{top_k} Recall: {round(np.mean(recall),2)*100}")
                    
                print()
                eval_results_filename = f"eval_results_{epoch+1}.json"
                eval_results_file_path = os.path.join(output_dir, eval_results_filename)
                print(f"Saving eval results to {eval_results_file_path}")
                with open(eval_results_file_path, "w") as f_obj:
                    json.dump(eval_results, f_obj, indent=4)
                model_filename = f"model_{epoch+1}.pt"
                model_file_path = os.path.join(output_dir, model_filename)
                print(f"Saving model to {model_file_path}")
                torch.save(model, model_file_path)
                print()

class ARGS:
  def __init__(self,exp_name, rand_seed, batch_size, checkpoint_dir,output_dir,data_dir):
    self.exp_name = exp_name
    self.rand_seed = rand_seed
    self.batch_size = batch_size
    self.checkpoint_dir = checkpoint_dir
    self.output_dir = output_dir
    self.data_dir = data_dir

if __name__ == "__main__":
    args = ARGS('testing',0,64,'drive/MyDrive/WatClaimCheck/paper_code/checkpoint', 
                'drive/MyDrive/WatClaimCheck/paper_code/output', '/content/drive/MyDrive/WatClaimCheck/paper_code/dataset')
    print(pd.__version__)
    main(args)


1.3.5


Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializin


Epoch # 1 Train Loss: 25.25 Correct Predictions: 20.00

Warning! Could not save model for epoch: 1!

Epoch # 2 Train Loss: 3.25 Correct Predictions: 20.00

Warning! Could not save model for epoch: 2!

Epoch # 3 Train Loss: 1.76 Correct Predictions: 20.00

Warning! Could not save model for epoch: 3!

Epoch # 4 Train Loss: 1.93 Correct Predictions: 20.00

Warning! Could not save model for epoch: 4!

Epoch # 5 Train Loss: 1.85 Correct Predictions: 20.00

Warning! Could not save model for epoch: 5!

Epoch # 6 Train Loss: 1.82 Correct Predictions: 20.00

Warning! Could not save model for epoch: 6!

Epoch # 7 Train Loss: 6.81 Correct Predictions: 20.00

Warning! Could not save model for epoch: 7!

Epoch # 8 Train Loss: 9.70 Correct Predictions: 20.00

Warning! Could not save model for epoch: 8!

Epoch # 9 Train Loss: 2.15 Correct Predictions: 20.00

Warning! Could not save model for epoch: 9!

Epoch # 10 Train Loss: 9.37 Correct Predictions: 20.00

Warning! Could not save model for epoch: 1

100%|██████████| 1/1 [00:00<00:00,  8.38it/s]


valid
Top-10 Recall: 50.0
Top-25 Recall: 100.0
Top-50 Recall: 100.0
Top-100 Recall: 100.0
test
Top-10 Recall: 67.0
Top-25 Recall: 100.0
Top-50 Recall: 100.0
Top-100 Recall: 100.0

Saving eval results to drive/MyDrive/WatClaimCheck/paper_code/output/testing/Seed_0/eval_results_10.json
Saving model to drive/MyDrive/WatClaimCheck/paper_code/output/testing/Seed_0/model_10.pt


Epoch # 11 Train Loss: 9.42 Correct Predictions: 20.00

Warning! Could not save model for epoch: 11!

Epoch # 12 Train Loss: 2.06 Correct Predictions: 0.00

Warning! Could not save model for epoch: 12!

Epoch # 13 Train Loss: 3.95 Correct Predictions: 20.00

Warning! Could not save model for epoch: 13!

Epoch # 14 Train Loss: 2.52 Correct Predictions: 20.00

Warning! Could not save model for epoch: 14!

Epoch # 15 Train Loss: 2.39 Correct Predictions: 20.00

Warning! Could not save model for epoch: 15!

Epoch # 16 Train Loss: 1.61 Correct Predictions: 20.00

Warning! Could not save model for epoch: 16!

Epoch # 17 Tr

...

100%|██████████| 1/1 [00:00<00:00,  7.21it/s]


valid
Top-10 Recall: 50.0
Top-25 Recall: 100.0
Top-50 Recall: 100.0
Top-100 Recall: 100.0
test
Top-10 Recall: 67.0
Top-25 Recall: 100.0
Top-50 Recall: 100.0
Top-100 Recall: 100.0

Saving eval results to drive/MyDrive/WatClaimCheck/paper_code/output/testing/Seed_0/eval_results_390.json
Saving model to drive/MyDrive/WatClaimCheck/paper_code/output/testing/Seed_0/model_390.pt


Epoch # 391 Train Loss: 1.60 Correct Predictions: 20.00

Warning! Could not save model for epoch: 391!

Epoch # 392 Train Loss: 1.86 Correct Predictions: 20.00

Warning! Could not save model for epoch: 392!

Epoch # 393 Train Loss: 1.83 Correct Predictions: 20.00

Warning! Could not save model for epoch: 393!

Epoch # 394 Train Loss: 1.67 Correct Predictions: 20.00

Warning! Could not save model for epoch: 394!

Epoch # 395 Train Loss: 1.69 Correct Predictions: 20.00

Warning! Could not save model for epoch: 395!

Epoch # 396 Train Loss: 1.53 Correct Predictions: 40.00

Warning! Could not save model for epoch: 396!

100%|██████████| 1/1 [00:00<00:00,  6.89it/s]


valid
Top-10 Recall: 50.0
Top-25 Recall: 100.0
Top-50 Recall: 100.0
Top-100 Recall: 100.0
test
Top-10 Recall: 67.0
Top-25 Recall: 100.0
Top-50 Recall: 100.0
Top-100 Recall: 100.0

Saving eval results to drive/MyDrive/WatClaimCheck/paper_code/output/testing/Seed_0/eval_results_400.json
Saving model to drive/MyDrive/WatClaimCheck/paper_code/output/testing/Seed_0/model_400.pt


RuntimeError: ignored

In [ ]:

#check python version
print(pd.__version__)

1.3.5
